In [1]:
import pandas as pd
from pathlib import Path
import uuid

In [2]:
fincausal_task1_train_path = Path('/media/sarthak/HDD/data_science/fnp_resources/data/task1/iteration_4/train.csv')
causaly_all_data_path = Path('/media/sarthak/HDD/data_science/fnp_resources/data/extra/causaly.csv')

In [3]:
causaly_df = pd.read_csv(causaly_all_data_path)
causaly_df.head()

,sentence,causal
0,"Results indicated that boys with ADHD , relati...",1
1,Ticagrelor appears to provide higher value for...,0
2,"Whatever the mechanism , active smoking is an ...",1
3,"In this study , we examined whether use of an ...",0
4,"Using causal inference testing , we searched c...",0


In [4]:
causaly_df.shape

(2000, 2)

In [5]:
fincausal_task1_train_df = pd.read_csv(fincausal_task1_train_path)
fincausal_task1_train_df.head()

,Index,Text,Gold,unique_id
0,268.00017,"In turn, the Liberal promise could benefit th...",0,5e9f149f-e1c3-43b8-a4f0-2614ca2bfd42
1,357.00082,From foundation modules that introduce asset ...,0,694a6824-613c-4978-9e17-a45b77e2b485
2,471.00003,The law firm Kessler Topaz Meltzer & Check LL...,0,661e6bf7-ffbf-4581-acff-8c246ecac51a
3,320.00001,News The UK Civil Aviation Authority has lau...,0,a6a80d24-dbc0-404d-9f71-e907524e88ec
4,521.00016,The proportion of middle-skill jobs for worke...,0,9657941b-aae0-4a9c-b1cd-8c7433f3b0bf


In [6]:
fincausal_task1_train_df.shape

(15576, 4)

In [7]:
# what's the distribution of label?
causal_distribution = fincausal_task1_train_df.Gold.value_counts() / len(fincausal_task1_train_df)
causal_distribution

0    0.927902
1    0.072098
Name: Gold, dtype: float64

In [8]:
# how many causal can I take from semeval?
causaly_noncausal = causaly_df.loc[causaly_df.causal == 0]
causaly_causal = causaly_df.loc[causaly_df.causal == 1]

causaly_causal_resampled = causaly_causal.sample(n=int((causal_distribution[1]*100*len(causaly_noncausal))/(100-(causal_distribution[1]*100))))

causaly_resampled = pd.concat([causaly_noncausal, causaly_causal_resampled])
causaly_resampled.causal.value_counts() / len(causaly_resampled)

0    0.928796
1    0.071204
Name: causal, dtype: float64

In [9]:
# combine both the datasets
causly_resampled_formatted_df = pd.DataFrame({
    'Index': None,
    'Text': causaly_resampled.sentence,
    'Gold': causaly_resampled.causal,
    'unique_id': [str(uuid.uuid4()) for _ in range(len(causaly_resampled))]
})
causly_resampled_formatted_df.head()

,Index,Text,Gold,unique_id
1,None,Ticagrelor appears to provide higher value for...,0,8e98162c-fe97-4ec2-b02e-a7984f172e9b
3,None,"In this study , we examined whether use of an ...",0,66703998-091c-4ccc-acc0-402befa976b3
4,None,"Using causal inference testing , we searched c...",0,9b959951-152d-477d-9cbb-4b5048d6f2a5
6,None,Data were collected on the incidence of POAF l...,0,a15b3421-48da-4a07-b994-c23dd0111e11
8,None,"In the current study , women were losing BMC ,...",0,f7149a4f-0b31-49bd-ad41-88aacb671d7f


In [10]:
causly_resampled_formatted_df.Gold.value_counts() / len(causly_resampled_formatted_df)

0    0.928796
1    0.071204
Name: Gold, dtype: float64

In [11]:
fincausal_task1_train_combined_causaly = pd.concat([fincausal_task1_train_df, causly_resampled_formatted_df])
fincausal_task1_train_combined_causaly.shape

(16531, 4)

In [12]:
fincausal_task1_train_combined_causaly.Gold.value_counts() / len(fincausal_task1_train_combined_causaly)

0    0.927954
1    0.072046
Name: Gold, dtype: float64

In [13]:
fincausal_task1_train_combined_causaly.to_csv('/media/sarthak/HDD/data_science/fnp_resources/data/task1/iteration_4/train_with_causaly.csv', index=False)